# Estimate the ratio of Pb to black carbon for atmospheric deposition parameterization

Using Alert, Nunavut observations (Sharma et al., 2019) accessed through the World Database for Aerosols (https://www.gaw-wdca.org/). https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019JD030844

Pb data:
- instrument type: high_vol_sampler (CA01L_HVS420_HM_MC) 
- frequency: 1w
- dates available: 1980-07-19 to 1995-05-22
    
Equivalent Black Carbon (EBC) data
- instrument type: filter_absorption_photometer (CA05L_Magee_Model_AE6) 
- frequency: 1h
- dates available: 1989-05-29 to 1992-12-31, 2011-01-01 to 2012-12-29

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib.dates import num2date
import matplotlib.dates as mdates
import datetime
import calendar
from scipy.optimize import leastsq
from scipy.stats import pearsonr

%matplotlib inline

#### Load data

In [2]:
# Load observations of EBC and Pb from Alert, downloaded from the World Database for Aerosols
Alert           = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/Alert_1989-1992.nc')
Alert_EBC       = Alert['EBC'].values*1e3 # ng/m3
Alert_EBC_dates = num2date(Alert['EBC_dates'].values)
Alert_Pb        = Alert['Pb'].values
Alert_Pb_dates  = num2date(Alert['Pb_dates'].values)

#### Functions

In [3]:
# convert date to a timestamp
def toTimestamp(d):
    return calendar.timegm(d.timetuple())

In [5]:
# Function calculates a linear fit based on x and y data points; based off an example on stackoverflow
def linear_fit(x, y):

    fitfunc = lambda params, x: params[0] * x  
    errfunc = lambda p, x, y: fitfunc(p, x) - y  #create error function for least squares fit

    # calculate best fit parameters using the error function
    p1, success = leastsq(errfunc, np.array((0.5)), args = (x, y))
    fit         = fitfunc(p1, x) 
    
    return p1, fit  

In [6]:
# Function applies a boxcar filter to the observations with specified window length:
def boxcar_filter_data(variable, sample_frequency='7d', window_len='5d'):
   
    # Calculate boxcar window length:
    if (window_len[-1] == 'd'):
        if (sample_frequency[-1] == 'd'): # units of days
            window_length = int(window_len[:-1]) / int(sample_frequency[:-1])
        elif (sample_frequency[-1] == 'h'): # units of hours
            window_length = int(window_len[:-1])*24 / int(sample_frequency[:-1])            
        else:
            prin('Sampling_frequency can only be in units of h (hours) or d (days)')
    else:
        print('Boxcar window_len can only be in units of d (days)')
        
    # Smooth data using window length: (based off Scipy recipe)
    #   Convolution of a scaled window with the signal. The signal is prepared by introducing reflected copies of the signal 
    #   (with the window size) in both ends.
    window_len = int(window_length)
    s          = np.r_[variable[window_len-1:0:-1], variable, variable[-2:-window_len-1:-1]]
    w          = np.ones(window_len,'d') # moving average:
    boxcar     = np.convolve(w / w.sum(), s, mode='valid')
    
    ind = int(np.floor(window_length/2))
    
    return boxcar[ind:-ind+1]

In [7]:
# Resample observations to a regular time grid: 
def resample_data(dates, variable, time_step='7d'):
    datetime_dates = [pd.to_datetime(date) for date in dates]
        
    # resample / interpolate data before applying filters so that they are evenly spaced to time_step size in days    
    if time_step[-1] == 'd':
        # units of days
        n_steps       = np.floor((datetime_dates[-1] - datetime_dates[0]).days / float(time_step[:-1])) # Number of time steps to apply
        gridded_dates = np.array([datetime_dates[0] + datetime.timedelta(days=int(time_step[:-1])*n) for n in range(int(n_steps))])
    elif time_step[-1] == 'h':
        # units of hours
        n_steps       = np.floor((datetime_dates[-1] - datetime_dates[0]).days*24 / float(time_step[:-1])) # Number of time steps to apply
        gridded_dates = np.array([datetime_dates[0] + datetime.timedelta(hours=int(time_step[:-1])*n) for n in range(int(n_steps))])
    else:
        print("Function is only set up for time_step units in d (days) or h (hour)")

    # convert dates to timestamps for interpolation
    timestamp_dates         = np.array([toTimestamp(t) for t in datetime_dates])
    timestamp_gridded_dates = np.array([toTimestamp(t) for t in gridded_dates])
    
    # interpolate dates:
    variable_gridded = np.interp(timestamp_gridded_dates, timestamp_dates, variable)

    return gridded_dates, variable_gridded

In [8]:
# Match observations for Pb and EBC that are on the same dates
def match_dates(array1, array2, array1_dates, array2_dates):
    indx_array1 = np.array([])
    indx_array2 = np.array([])

    # Create arrays of the time periods when there are Pb and EBC data:
    for ind, date1 in enumerate(array1_dates):
        # Find index of nearest date for each measurement:
        difference = np.array([(date1 - date2).total_seconds() for date2 in array2_dates])
        indx_add = np.argmin(np.abs(difference))
        # Save indices to array:
        indx_array1  = np.append(indx_array1, int(ind))
        indx_array2  = np.append(indx_array2, int(indx_add))
        
    array1_dates_match = np.array([array1_dates[int(ix)] for ix in indx_array1])
    array1_match       = np.array([array1[int(ix)] for ix in indx_array1])
    array2_dates_match = np.array([array2_dates[int(ix)] for ix in indx_array2])
    array2_match       = np.array([array2[int(ix)] for ix in indx_array2])

    return array1_match, array2_match, array1_dates_match, array2_dates_match

#### Calculations

Subset to correct date range:

In [9]:
Alert_Pb_dates_subset  = Alert_Pb_dates[18:]
Alert_Pb_subset        = Alert_Pb[18:]
Alert_EBC_dates_subset = Alert_EBC_dates[:]
Alert_EBC_subset       = Alert_EBC[:]

Match nearest dates of measurements:

In [10]:
Alert_Pb_match, Alert_EBC_match, \
    Alert_Pb_dates_match, Alert_EBC_dates_match = match_dates(Alert_Pb_subset, Alert_EBC_subset, Alert_Pb_dates_subset, Alert_EBC_dates_subset)

Process / filter data:

In [11]:
# Grids are irregular, so interpolate to regular grid:
Alert_Pb_dates_gridded , Alert_Pb_gridded  = resample_data(Alert_Pb_dates_subset, Alert_Pb_subset, time_step='7d')   # Measured roughly weekly
Alert_EBC_dates_grid,    Alert_EBC_grid    = resample_data(Alert_EBC_dates_subset, Alert_EBC_subset, time_step='1h') # Measured roughly hourly

In [12]:
# Remove 1991-2-26 to 1991-3-11 from regridded dataset because no observations during this time period:
Alert_EBC_dates_gridded = Alert_EBC_dates_grid[(Alert_EBC_dates_grid < Alert_EBC_dates[11862]) \
                                                  | (Alert_EBC_dates_grid > Alert_EBC_dates[11863])]


Alert_EBC_gridded       = Alert_EBC_grid[(Alert_EBC_dates_grid < Alert_EBC_dates[11862]) \
                                                  | (Alert_EBC_dates_grid > Alert_EBC_dates[11863])]

In [13]:
Alert_Pb_boxcar  = boxcar_filter_data(Alert_Pb_gridded , sample_frequency='7d', window_len='30d')
Alert_EBC_boxcar = boxcar_filter_data(Alert_EBC_gridded, sample_frequency='1h', window_len='30d')

Correlate Pb:BC

In [14]:
# Boxcar:
Alert_Pb_boxcar_match, Alert_EBC_boxcar_match, \
    Alert_Pb_dates_boxcar_match, Alert_EBC_dates_boxcar_match = match_dates(Alert_Pb_boxcar, Alert_EBC_boxcar, \
                                                                           Alert_Pb_dates_gridded, Alert_EBC_dates_gridded)

# Linear fit:
p1_boxcar_Alert, f_boxcar_Alert = linear_fit(Alert_EBC_boxcar_match, Alert_Pb_boxcar_match)
print(p1_boxcar_Alert, 1/p1_boxcar_Alert) # Pb = ratio*BC

[0.01089848] [91.75595298]


In [15]:
print('Correlation before any filtering is applied:')
print(f'Pearson r: {pearsonr(Alert_EBC_match, Alert_Pb_match)[0]:.2f}')

print('Correlation after boxcar filtering:')
print(f'Pearson r: {pearsonr(Alert_EBC_boxcar_match, Alert_Pb_boxcar_match)[0]:.2f}')

Correlation before any filtering is applied:
Pearson r: 0.54
Correlation after boxcar filtering:
Pearson r: 0.72
